<a href="https://colab.research.google.com/github/oownus1/AutoML/blob/main/05_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC(%ED%95%9C%EA%B8%80)_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 필요한 모듈

1. `konlpy`: 한글 형태소 분석기
  - [공식문서](https://konlpy.org/ko/latest/)
  - 한글 자연어처리를 위한 필수 설치 라이브러리

2. `pororo`: 카카오브레인에서 만든 한글 자연어처리 라이브러리
  - [Github 주소](https://github.com/kakaobrain/pororo)
  - [공식문서](https://kakaobrain.github.io/pororo/)

In [ ]:
#pororo란 -> 신경망 모델을 사용한 자연어 처리기
#pycaret을 사용하여 자연어 처리를 하게 되면 영어는 강하지만 한글은 약하다
#pororo는 한글 대상으로 할 수 있는 자연어 처리기 특징이 있다
#자동으로 에세이 측정, 리뷰, 감정 분석, 텍스트 유사도 분석, 토픽 분석, 빈칸 내용 채워주기 및 예측, 번역기, 텍스트 요약, 이미지 묘사하는 글귀 적기 등 다양한 기능들이 있다
#자연어 처리 모델이 굉장히 방대한 양의 데이터를 학습하였다

In [1]:
# konlpy 설치
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.1 MB/s 
     |████████████████████████████████| 453 kB 62.1 MB/s 


In [ ]:
# Pororo 설치
!pip install git+https://github.com/kakaobrain/pororo.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/kakaobrain/pororo.git to /tmp/pip-req-build-2zrd4066
  Running command git clone -q https://github.com/kakaobrain/pororo.git /tmp/pip-req-build-2zrd4066
     |████████████████████████████████| 748.8 MB 19 kB/s 
     |████████████████████████████████| 5.9 MB 2.7 MB/s 
     |████████████████████████████████| 11.0 MB 59.8 MB/s 
     |████████████████████████████████| 4.7 MB 56.5 MB/s 
     |████████████████████████████████| 85 kB 4.7 MB/s 
     |████████████████████████████████| 3.1 MB 50.3 MB/s 
     |████████████████████████████████| 468 kB 64.8 MB/s 
     |████████████████████████████████| 1.2 MB 67.2 MB/s 
     |████████████████████████████████| 42.4 MB 1.3 MB/s 
     |████████████████████████████████| 116 kB 62.9 MB/s 
     |████████████████████████████████| 123 kB 73.1 MB/s 
     |████████████████████████████████| 235 kB 69.7 MB/s 
     |███████████████████

In [ ]:
# mecab 형태소 분석기 설치
!pip install python-mecab-ko

In [ ]:
# 예제 실행을 위한 사전 import
from pororo import Pororo

# 빈칸 채우기
fib = Pororo(task="fib", lang="ko")

# 이미지 캡셔닝
caption = Pororo(task="caption", lang="en")

# 자동 에세이 채점
aes = Pororo(task="aes", lang="en")

# 기계번역
mt = Pororo(task="translation", lang="multi")

# 한글 맞춤법(띄어쓰기) 수정
spacing = Pororo(task="gec", lang="ko")

# 영어 알파벳 수정
gec = Pororo(task="gec", lang="en")

# 요약
bullet_summ = Pororo(task="text_summarization", lang="ko", model="bullet")
ext_summ = Pororo(task="text_summarization", lang="ko", model="extractive")
abs_summ = Pororo(task="text_summarization", lang="ko", model="abstractive")

## 형태소 분석기 `konlpy` 예제

In [ ]:
# 컴퓨터가 한글을 분석하려면 단어들을 숫자로 변경을 해주어야 된다
# 순서 : 1. 토큰화 -> 쪼개는 작업(글자단위로, 단어별로, 공백기준, 문장단위 등 다양한 방식으로 쪼갤 수 있다)
# 방탄소년단 + 이 이런식으로 내부적으로 추출한다.
input_text = '''그룹 방탄소년단(BTS)이 두 번째 영어 곡 '버터'(Butter)로 미국 빌보드 메인 싱글 차트에서 2주 연속 정상에 올랐습니다.
빌보드는 지난달 21일 발매된 BTS의 '버터'가 메인 싱글 차트인 '핫 100'에서 지난주에 이어 1위를 기록했다고 7일(현지시간) 밝혔습니다.
'''
input_text

In [ ]:
# 형태소 분석기 가져오기
from konlpy.tag import Okt

In [ ]:
# 객체 생성
# 태그를 가져와서 쪼갠다
okt = Okt()

In [ ]:
okt.pos(input_text)[:10]
#명사(Noun) 이런식으로 분리된 것이 나온다 -> 얘네를 단어별로 숫자형태로 변환하여 수치 변환 진행

In [ ]:
okt.morphs(input_text)[:20]

## 문서 요약 (Text Summerization)

- [문서링크](https://kakaobrain.github.io/pororo/seq2seq/summary.html)

샘플 데이터 생성

In [ ]:
input_text1 = """가수 김태연은 걸 그룹 소녀시대, 소녀시대-태티서 및 소녀시대-Oh!GG의 리더이자 메인보컬이다. 2004년 SM에서 주최한 청소년 베스트 선발 대회에서 노래짱 대상을 수상하며 SM 엔터테인먼트에 캐스팅되었다. 이후 3년간의 연습생을 거쳐 2007년 소녀시대의 멤버로 데뷔했다. 태연은 1989년 3월 9일 대한민국 전라북도 전주시 완산구에서 아버지 김종구, 어머니 김희자 사이의 1남 2녀 중 둘째로 태어났다. 가족으로는 오빠 김지웅, 여동생 김하연이 있다. 어릴 적부터 춤을 좋아했고 특히 명절 때는 친척들이 춤을 시키면 곧잘 추었다던 태연은 TV에서 보아를 보고 가수의 꿈을 갖게 되었다고 한다. 전주양지초등학교를 졸업하였고 전주양지중학교 2학년이던 2003년 SM아카데미 스타라이트 메인지방보컬과 4기에 들어가게 되면서 아버지와 함께 주말마다 전주에서 서울로 이동하며 가수의 꿈을 키웠다. 2004년에 당시 보컬 트레이너였던 더 원의 정규 2집 수록곡 〈You Bring Me Joy (Part 2)〉에 피처링으로 참여했다. 당시 만 15세였던 태연은 현재 활동하는 소속사 SM 엔터테인먼트에 들어가기 전이었다. 이후 태연은 2004년 8월에 열린 제8회 SM 청소년 베스트 선발 대회에서 노래짱 부문에 출전해 1위(대상)를 수상하였고 SM 엔터테인먼트에 정식 캐스팅되어 연습생 생활을 시작하게 되었다. 2005년 청담고등학교에 입학하였으나, 학교 측에서 연예계 활동을 용인하지 않아 전주예술고등학교 방송문화예술과로 전학하였고 2008년 졸업하면서 학교를 빛낸 공로로 공로상을 수상했다. 태연은 연습생 생활이 힘들어 숙소에서 몰래 뛰쳐나갔다가 하루 만에 다시 돌아오기도 했다고 이야기하기도 했다. 이후 SM엔터테인먼트에서 3년여의 연습생 기간을 거쳐 걸 그룹 소녀시대의 멤버로 정식 데뷔하게 되었다."""
input_text2 = """목성과 토성이 약 400년 만에 가장 가까이 만났습니다. 국립과천과학관 등 천문학계에 따르면 21일 저녁 목성과 토성은 1623년 이후 397년 만에 가장 가까워졌는데요. 크리스마스 즈음까지 남서쪽 하늘을 올려다보면 목성과 토성이 가까워지는 현상을 관측할 수 있습니다. 목성의 공전주기는 11.9년, 토성의 공전주기는 29.5년인데요. 공전주기의 차이로 두 행성은 약 19.9년에 한 번 가까워집니다. 이번 근접 때  목성과 토성 사이 거리는 보름달 지름의 5분의 1 정도로 가까워졌습니다. 맨눈으로 보면 두 행성이 겹쳐져 하나의 별처럼 보이는데요. 지난 21일 이후 목성과 토성의 대근접은 2080년 3월 15일로 예측됩니다. 과천과학관 측은 우리가 대근접을 볼 수 있는 기회는 이번이 처음이자 마지막이 될 가능성이 크다라고 설명했 습니다."""

In [ ]:
input_text1

In [ ]:
# supports various decoding strategies 
# abs_summ -> 문서 요약해준다.
abs_summ(
    input_text1, #데이터 넣어줌
    beam=5,      #어느정도 범위까지 볼것인지
    len_penalty=0.6,  
    no_repeat_ngram_size=3,
    top_k=50,     #단어 최대 몇개까지 빈도수 가져갈것인지
    top_p=0.7
) #기본값 그대로 두었다

In [ ]:
input_text2

In [ ]:
# supports various decoding strategies 
abs_summ(
    input_text2, 
    beam=5, 
    len_penalty=0.6,
    no_repeat_ngram_size=3,
    top_k=50,
    top_p=0.7
)

### 3개의 주요 문장 추출

In [ ]:
# ext_summ 은 3문장으로 요약해줌
output_text1 = ext_summ(input_text1)
for line in output_text1.split('.'):
    print(line)

In [ ]:
output_text2 = ext_summ(input_text2)
for line in output_text2.split('.'):
    print(line)

### Bullet 포인트

In [ ]:
# supports various decoding strategies 
# 핵심이 되는 단어로 요약해줌
bullet_summ(
    input_text1, 
    beam=5, 
    len_penalty=0.6,
    no_repeat_ngram_size=3,
    top_k=50,
    top_p=0.7
)

### 신문기사 요약

In [ ]:
# 신문기사의 URL
# 크롤링으로 이 신문기사 본문만 발췌해온 다음에 이를 자연어 처리 모델에 넣고 신문기사를 요약해보는 실습
url = 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=101&oid=009&aid=0004886133'

뉴스 기사 내용 받아오기

In [ ]:
from bs4 import BeautifulSoup #크롤링 관련 라이브러리
import requests

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
req = requests.get(url, headers=headers)
soup = BeautifulSoup(req.text, 'html.parser')
body = soup.find_all(id='newsct_article') # 기사url 가서 f12누르면 위의 사각형 클릭하고 가져오고 싶은 상태에 마우스 커서 두고 코드 클릭해보면서 영역 확인-> 여기있는 id 값을 가져와서 바꿔준다=> 이 아이디 값이 자주 바뀐다.
body[0].text 

In [ ]:
abs_summ(
    body[0].text, 
    beam=5, 
    len_penalty=0.6,
    no_repeat_ngram_size=3,
    top_k=50,
    top_p=0.7
)

In [ ]:
def summerize_news(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    body = soup.find_all(id='newsct_acticle')
    if len(body) > 0:
        text = body[0].text
        output_bullet = bullet_summ(text, 
                                    beam=5, 
                                    len_penalty=0.6,
                                    no_repeat_ngram_size=3,
                                    top_k=50,
                                    top_p=0.7)
        summary_output = abs_summ(text, 
                                  beam=5, 
                                  len_penalty=0.6,
                                  no_repeat_ngram_size=3,
                                  top_k=50,
                                  top_p=0.7
                                  )
        print('[주요]')
        for line in output_bullet:
            print(f'- {line}')
        print('[요약]')
        print(summary_output)
    else:
        return "에러ㅜ"

In [ ]:
summerize_news('https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=101&oid=009&aid=0004886133')

## 맞춤법 수정 (Grammar Correction)

In [ ]:
#문서 수정 시 오류 나는 것 자동으로 해준다.

### 문법 오류 수정

In [ ]:
# 오류 문장 입력
gec("This apple are so sweet.")

### 한글 띄어쓰기 오류 수정

In [ ]:
# 오류 문장 입력
spacing("아버지가방에들어간다.")

## 기계번역 Machine Translation

In [ ]:
# mt = Pororo(task="translation", lang="multi")
# 한글을 영어로 해석 
# 다른 나라 언어로도 해석이 가능하지만 아직은 기능이 약하다고 합니다.
mt("케빈은 아직도 일을 하고 있다.", src="ko", tgt="en")

## 자동 에세이 채점 Automated Essay Scoring

In [ ]:
# aes = Pororo(task="aes", lang="en")
aes('''
The lecturer talks about research conducted by a firm that used the group system to handle their work. He says that the theory stated in the passage was very different and somewhat inaccurate when compared to what happened for real.
First, some members got free rides. That is, some didn’t work hard but gotrecognition for the success nontheless. This also indicates that people who worked hard was not given recognition they should have got. In other words, they weren’t given the oppotunity to “shine”. This derectly contradicts what the passage indicates.
Second, groups were slow in progress. The passage says that groups are nore responsive than individuals because of the number of people involved and their aggregated resources. However, the speaker talks about how the firm found out that groups were slower than individuals in dicision making. Groups needed more time for meetings, which are neccesary procceedures in decision making. This was another part where experience contradicted theory.
Third, influetial people might emerge, and lead the group towards glory or failure. If the influent people are going in the right direction there would be no problem. But in cases where they go in the wrong direction, there is nobody that has enough influence to counter the decision made. In other words, the group might turn into a dictatorship, with the influential party as the leader, and might be less flexible in thinking. They might become one-sided, and thus fail to succeed.
''')

In [ ]:
aes('''
Throw out the bottles and boxes of drugs in your house. A new theory suggests that medicine could be bad for your health, which should at least come as good news to people who cannot afford to buy expensive medicine. However, it is a blow to the medicine industry, and an even bigger blow to our confidence in the progress of science. This new theory argues that healing is at our fingertips: we can be healthy by doing Reiki on on a regular basis.
''')

In [ ]:
aes('''
Online games aren't just a diversion, but a unique way to meet other people. As millions of gamers demonstrate, playing online is about friendship and cooperation, not just killing monsters. These games are a viable social network because players focus on teamwork, form groups with like-minded people and have romantic relationships with other players. Massively Multiplayer Online Games (MMOGs) feature millions of players interacting in the same environment. The games are social in nature as they allow players to band together and complete missions based on a story line, or test their skills by fighting against each other. At the start of the game, the user creates a fictional character, and customizes its physical appearance. Since many games involve combat, players also outfit their characters with armor and weapons, as well as choose their "profession." Many popular game titles like World of Warcraft and Everquest follow a fantasy theme, so most professions have magical abilities like healing other players or raising undead minions. While the process seems simple, players may spend hours agonizing over the perfect look for their character, from their armor color to the type of skills to use in battle. Once their character is created, the player is free to explore the vast, digital world and interact with other players; however they must pay on average $15 a month for game content. MMOG users are mostly male - usually between the ages of 18-34 - although titles like World of Warcraft have a healthy population of female players as well. With millions of players, there are plenty of people to adventure with.
''')

## Fill in the blank

In [ ]:
fib("손흥민은 __의 축구선수이다.")

In [ ]:
fib("대한민국의 수도는 __이다.")

## 이미지 캡션 (Image Captioning)

In [ ]:
from IPython.display import Image

In [ ]:
url = 'https://i.pinimg.com/originals/b9/de/80/b9de803706fb2f7365e06e688b7cc470.jpg'
Image(url, width=750)

In [ ]:
caption(url)